In [1]:
import findspark

In [2]:
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import  *

In [4]:
spark = SparkSession.builder \
.appName("DataframeWriter ORC") \
.master("yarn") \
.enableHiveSupport() \
.getOrCreate()

In [5]:
df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.option("compression","gzip") \
.load("/user/train/datasets/Hotel_Reviews.csv.gz")

In [6]:
df2 = df.withColumn("Tags", 
                     F.split(F.col("Tags"), ",")
                     .cast(ArrayType(StringType()))) \
.withColumn("Review_Date", F.to_date(F.col("Review_Date"),"M/d/yyyy"))

In [7]:
df2.limit(2).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[[' Leisure trip ', ' Couple ', ' Duplex Dou...",0 days,52.3605759,4.9159683


In [8]:
df2.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [9]:
import time

# orc with default snappy compression

In [10]:

start_time = time.time()

df2.write \
.format("orc") \
.mode("overwrite") \
.save("hdfs://localhost:9000/user/train/output_data/hotel_review_orc")


print("----------- %s secs ----------" %(time.time() - start_time))

----------- 39.451637506484985 secs ----------


In [11]:
! hdfs dfs -ls -h /user/train/output_data/hotel_review_orc

Found 2 items
-rw-r--r--   1 train supergroup          0 2022-01-02 00:20 /user/train/output_data/hotel_review_orc/_SUCCESS
-rw-r--r--   1 train supergroup     55.6 M 2022-01-02 00:20 /user/train/output_data/hotel_review_orc/part-00000-9d0000ed-5edf-4a8c-85c1-baee20cbcb73-c000.snappy.orc


# orc with compression

In [17]:
start_time = time.time()

df2.write \
.format("orc") \
.mode("overwrite") \
.option("compression","zlib") \
.save("hdfs://localhost:9000/user/train/output_data/hotel_review_orc_compress")


print("----------- %s secs ----------" %(time.time() - start_time))

----------- 35.241947412490845 secs ----------


In [18]:
! hdfs dfs -ls -h /user/train/output_data/hotel_review_orc_compress

Found 2 items
-rw-r--r--   1 train supergroup          0 2022-01-02 00:23 /user/train/output_data/hotel_review_orc_compress/_SUCCESS
-rw-r--r--   1 train supergroup     40.5 M 2022-01-02 00:23 /user/train/output_data/hotel_review_orc_compress/part-00000-574c5a98-9761-4045-8fce-a7ad80087872-c000.zlib.orc
